In [20]:
from glob import glob
from os.path import splitext
import PIL.Image as Image

pnglist = glob( "/home/patrick/HW3/cartoonset10k/cartoonset10k/*.[pP][nN][gG]" )

i=0
for png in pnglist:
    im = Image.open(png)
    i=i+1
    im.save('/home/patrick/HW3/cartoonset10k/cartoon/%05d'%(i)+'.png')

In [45]:
import os
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import save_image
import time

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
image_size = 64*64*3
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 100
learning_rate = 1e-3

# Create a directory if not exists
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [46]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import random_split

transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = torchvision.datasets.ImageFolder('/home/patrick/HW3/cartoonset10k', transform) 

data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

#print(len(dataset))

In [51]:
# VAE model
start_time = time.time()
class VAE(nn.Module):
    def __init__(self, image_size=64*64*3, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return torch.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
v_loss  = []
# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # Compute reconstruction loss and kl divergence
        reconst_loss = F.mse_loss(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        v_loss.append(loss.data[0])
        
        if (i+1) % 10 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 3, 64, 64)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 3, 64, 64), out.view(-1, 3, 64, 64)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))
        
end_time = time.time()
print('Total cost time',
      time.strftime("%H hr %M min %S sec", time.gmtime(end_time - start_time)))

/home/god/python3-virtual/lib/python3.5/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/home/god/python3-virtual/lib/python3.5/site-packages/ipykernel_launcher.py:51: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch[1/15], Step [10/100], Reconst Loss: 117206.2031, KL Div: 12764.6045
Epoch[1/15], Step [20/100], Reconst Loss: 100950.1875, KL Div: 6755.6099
Epoch[1/15], Step [30/100], Reconst Loss: 80727.9844, KL Div: 6413.7520
Epoch[1/15], Step [40/100], Reconst Loss: 73722.4141, KL Div: 4646.9712
Epoch[1/15], Step [50/100], Reconst Loss: 71205.9844, KL Div: 5098.8652
Epoch[1/15], Step [60/100], Reconst Loss: 62149.2617, KL Div: 5194.8296
Epoch[1/15], Step [70/100], Reconst Loss: 58546.2266, KL Div: 4566.7627
Epoch[1/15], Step [80/100], Reconst Loss: 59824.1719, KL Div: 4854.9482
Epoch[1/15], Step [90/100], Reconst Loss: 64929.7148, KL Div: 4915.0020
Epoch[1/15], Step [100/100], Reconst Loss: 64174.3633, KL Div: 5398.9111
Epoch[2/15], Step [10/100], Reconst Loss: 49370.8320, KL Div: 4653.7041
Epoch[2/15], Step [20/100], Reconst Loss: 46043.3867, KL Div: 4943.3232
Epoch[2/15], Step [30/100], Reconst Loss: 49877.7383, KL Div: 5015.3340
Epoch[2/15], Step [40/100], Reconst Loss: 57671.3555, KL Div

Epoch[12/15], Step [50/100], Reconst Loss: 32141.1797, KL Div: 3746.0681
Epoch[12/15], Step [60/100], Reconst Loss: 38562.0625, KL Div: 3746.6362
Epoch[12/15], Step [70/100], Reconst Loss: 37936.4805, KL Div: 3706.3892
Epoch[12/15], Step [80/100], Reconst Loss: 33717.1562, KL Div: 3386.9714
Epoch[12/15], Step [90/100], Reconst Loss: 34492.2422, KL Div: 3700.0610
Epoch[12/15], Step [100/100], Reconst Loss: 35933.1406, KL Div: 3702.3079
Epoch[13/15], Step [10/100], Reconst Loss: 34117.2227, KL Div: 3475.5972
Epoch[13/15], Step [20/100], Reconst Loss: 32748.1289, KL Div: 3481.0027
Epoch[13/15], Step [30/100], Reconst Loss: 36853.4922, KL Div: 3721.7722
Epoch[13/15], Step [40/100], Reconst Loss: 35877.4922, KL Div: 3787.7500
Epoch[13/15], Step [50/100], Reconst Loss: 32790.5938, KL Div: 3614.7427
Epoch[13/15], Step [60/100], Reconst Loss: 33353.2930, KL Div: 3596.2979
Epoch[13/15], Step [70/100], Reconst Loss: 35612.8672, KL Div: 3758.5723
Epoch[13/15], Step [80/100], Reconst Loss: 40797.3

In [52]:
import matplotlib.pyplot as plt

# plot learning curve
plt.plot(v_loss, label="loss")
plt.xlabel("iterations")
plt.ylabel("loss")
plt.savefig("loss")
plt.close()